In [1]:
import os

In [2]:
%pwd

'/home/mahesh/Regression_ML_Wine/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/mahesh/Regression_ML_Wine'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size

In [10]:
pip install requests

  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.8.3-py3-none-any.whl.metadata (2.4 kB)
Using cached requests-2.32.4-py3-none-any.whl (64 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached urllib3-2.5.0-py3-none-any.whl (129 kB)
Using cached certifi-2025.8.3-py3-none-any.whl (161 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [requests]━━ 4/5 [requests]
Note: you may need to restart the kernel to use updated packages.


In [19]:
import os
import zipfile
import requests
from pathlib import Path
from mlProject import logger  # adjust to your logger module
from src.utils.common import get_size  # adjust if needed
from src.config.configuration import DataIngestionConfig  # your config entity


class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        """
        Downloads a zip file from the source_URL if it doesn't already exist locally.
        Saves debug file if the content is not a zip.
        """
        local_file_path = self.config.local_data_file

        if not os.path.exists(local_file_path):
            logger.info(f"Downloading from: {self.config.source_URL}")
            response = requests.get(self.config.source_URL, stream=True)
            content_type = response.headers.get("Content-Type", "")
            logger.info(f"Response status: {response.status_code}")
            logger.info(f"Content-Type: {content_type}")

            # Save raw response for debugging
            with open("download_debug.html", "wb") as f:
                f.write(response.content)

            if response.status_code == 200 and 'zip' in content_type:
                with open(local_file_path, "wb") as f:
                    for chunk in response.iter_content(chunk_size=8192):
                        f.write(chunk)
                logger.info(f"File downloaded and saved at: {local_file_path}")
            else:
                raise ValueError(
                    f"Failed to download a valid ZIP file.\n"
                    f"Status: {response.status_code}\n"
                    f"Content-Type: {content_type}\n"
                    f"Inspect 'download_debug.html' for more details."
                )
        else:
            logger.info(f"File already exists: {local_file_path} (Size: {get_size(Path(local_file_path))})")

    def extract_zip_file(self):
        """
        Extracts the zip file into the specified directory.
        Raises error if file is not a valid zip.
        """
        unzip_path = self.config.unzip_dir
        local_file_path = self.config.local_data_file

        os.makedirs(unzip_path, exist_ok=True)

        try:
            with zipfile.ZipFile(local_file_path, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)
            logger.info(f"Extracted zip file to: {unzip_path}")
        except zipfile.BadZipFile:
            raise ValueError(f"File '{local_file_path}' is not a valid zip file.")


ModuleNotFoundError: No module named 'src.utils'

In [20]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)

    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    logger.error(f"Data ingestion failed: {e}")
    raise


[2025-08-07 20:11:30,415: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-08-07 20:11:30,419: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-07 20:11:30,432: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-08-07 20:11:30,435: INFO: common: created directory at: artifacts]
[2025-08-07 20:11:30,438: INFO: common: created directory at: artifacts/data_ingestion]
[2025-08-07 20:11:30,441: INFO: 131818154: File already exists at artifacts/data_ingestion/data.zip (size: ~ 177 KB)]
[2025-08-07 20:11:30,443: ERROR: 3925522071: Data ingestion failed: File 'artifacts/data_ingestion/data.zip' is not a valid zip file.]


ValueError: File 'artifacts/data_ingestion/data.zip' is not a valid zip file.